# Einstieg Python

## 1. Import der benötigten Module:

In [ ]:
import geopandas as gp
import pandas as pd
import matplotlib.pyplot as plt
import folium
import warnings

Warnungen können erst einmal unterdrückt werden

In [ ]:
warnings.filterwarnings('ignore')

## 2. Daten einladen und explorieren 

Laden Sie die *Berliner Bezirksdaten (Polygone)* und die *Berliner Bars (Punkte von OpenStreetMap extrahiert, s. Seminarblock 4)* ein.

In [ ]:
districts = gp.read_file("./data/bezirke_EPSG32633.shp")
bars = gp.read_file("./data/bars_bln.gpkg")

Lassen Sie sich die Attributtabelle des *Bezirke-Layers* über den *print*-Befehl anzeigen:

In [ ]:
print('Das ist der Bezirke GeoDataFrame.')
print(districts)

Schauen Sie sich die *geometry-Spalte* genauer an. Welche Informationen werden hier gespeichert?



<details>

<summary> <b>Antwort - klicken zum ausklappen</b></summary>
    
Die <i>*geometry-Spalte*</i> beinhaltet sowohl Informationen zu den Geometrien (Punkte, Linien oder Polygone) der einzelnen Elemente, als auch Informationen zum gespeichterten Koordinatensystem.

</details>


Nutzen Sie den *print-Befehl*, um sich die Attributtabelle vom *Bars-Layer* anzeigen zu lassen.

Können Sie aus dem Output herausfinden, wie viele Zeilen und Spalten der *Bars-Layer* hat?

In [ ]:
print('Das ist der Bar GeoDataFrame.')
print(bars)

Um einen besseren Überblick zu bekommen, welche Spalten in einem Datensatz enthalten sind (ähnlich zu den Spalten von *Attributtabellen* in *QGIS*), *printen* Sie sich lediglich die Spalten (columns). Erläuterungen zu den einzelnen Spalten bekommen sie [hier](https://wiki.openstreetmap.org/wiki/Map_features#Additional_properties). 

In [ ]:
print('Das sind alle Spalten des Bar-GeoDataFrames.')
print(bars.columns.tolist())

Wir konzentrieren uns für die nächsten Schritte nur auf die Spalten 'full_id', 'name' und 'geometry'. Wir selektieren diese Spalten und speichern diese in einer neuen Variable .

In [ ]:
bars_selected = bars[['full_id', 'name', 'geometry']]
print(bars_selected)

## 3. Koordinatenbezugssysteme

In einem nächsten Schritt wollen wir die die Koordinatensysteme der Datensätze bestimmen und ggf. reprojizieren.

In [ ]:
print('Das ist das CRS vom Bezirke-Layer:')
print(districts.crs)

In [ ]:
print('Das ist das CRS vom Bar-Layer:')
print(bars_selected.crs)

Liegen die beiden Datensätze im selben Koordinatenystem vor? Suchen sie weitere Informationen zu den Koordinatensystem, z.B. unter https://epsg.io/. 

<details>

<summary> <b>Antwort - klicken zum ausklappen</b></summary>
    
Die beiden Datensätze haben unterschiedliche Koordinatensysteme: 
    
Der Bezrike-Layer liegt im Koordinatensystem <a href="https://epsg.io/32633">WGS 84 / UTM zone 33N</a> vor. 
    
Der Bar-Layer liegt im Koordinatensystem <a href="https://epsg.io/4326">World Geodetic System 1984</a> vor.


</details>

Reprojizieren Sie das Koordinatenystem des *Bar-Layers* in das Koordinatenystem vom *Bezirke-Layer* mit der Funktion *to_crs()* und speichern das Ergebnis als neue Variable.

In [ ]:
bars_selected_reproj = bars_selected.to_crs(districts.crs)
print(bars_selected_reproj.crs)

## 4. statische Visualisierung

Sie können sich die Datensätze für einen schnellen Überblick visualisieren lassen. 

In [ ]:
# Bezirke-Layer
districts.plot()
plt.show()

In [ ]:
# Bar-Layer
bars_selected_reproj.plot()
plt.show()

Sie können auch mehrere Datensätze in einer Karte abbilden. Auch lassen sich Farben, Größen, Umrisse etc. verändern. Probieren Sie gerne weitere Darstellungsoptionen aus.

In [ ]:
base = districts.plot(color='white', edgecolor='black')

bars_selected_reproj.plot(ax=base, marker='o', color='red', markersize=3)

## 4. Spatial Join

Nun wollen wir herausfinden, **wie viele Bars es pro Bezirk gibt**. 

Dazu führen Sie in einem ersten Schritt einen *Spatial Join* durch. *Spatial Joins* dienen dazu, zwei Datensätze räumlich zu verknüpfen. Attribute werden aus dem ersten Datensatz an die Attribute im zweiten Datensatz auf Grundlage der relativen räumlichen Beziehung zwischen den Geometrien der beiden Datensätze angefügt. Eine Erklärung der verschiedenen Arten von relativen räumlichen Beziehung finden Sie [hier](https://pygis.io/docs/e_spatial_joins.html). Schauen Sie sich an, was die unterschiedlichen relativen räumlichen Beziehung (=Predicates) bedeuten (intersects, contains, equals, touches, overlaps, within, crosses).

In *QGIS* vollzieht man den folgenden *Spatial Join* über das Tool *Join Attributes by Location* aus der *Processing Toolbox*.


In [ ]:
bars_and_districts = districts.sjoin(bars_selected_reproj, predicate = 'contains')
print(bars_and_districts)

Schauen Sie sich das Ergebnis genau an. Was ist passiert?

Nun wollen wir in einem nächsten Schritt berechnen, wie viele Bars es pro Bezirk gibt. Dazu nutzen wir die Funktionen *groupby()* und *size()*.

In [ ]:
count_bars_per_district = bars_and_districts.groupby('BEZIRK_NAM').size().rename('count_per_district')
print(count_bars_per_district)

Nun wollen wir das Ergebnis wieder mit unserem ursprünglichen *Bezirke-Layer* zusammenführen. Dafür nutzen wir die Funktion *merge* aus dem Modul *Pandas*.

In [ ]:
districts_with_bars = pd.merge(districts, count_bars_per_district, how='inner', on=['BEZIRK_NAM'])

Wir können uns die Ergebnisse auch in einem Barplot anzeigen lassen: 

In [ ]:
plt.xticks(rotation='vertical')
plt.bar(districts_with_bars.BEZIRK_NAM, districts_with_bars.count_per_district)

## 5. interaktive Visualisierung

Zum Schluss wollen wir uns noch das Modul *Folium* anschauen. *Folium* ermöglicht uns mit wenige Zeilen Code eine **interaktive Web-Karte** zu erstellen. Dabei greift *Folium* auf die Open-Source JavaScript Bücherei [*Leaflet*](https://leafletjs.com/) drauf zu.

Zuerst müssen wir eine neue *ID-Spalte* einfügen.

In [ ]:
districts_with_bars["id"] = districts_with_bars.index.astype(str)

Jetzt können wir die interaktive Karte darstellen.

In [ ]:
# create base interactive map
interactive_map = folium.Map(
    location=(52.51791,13.39339),
    zoom_start=10
)

# define input layer
districts_with_bars_layer = folium.Choropleth(
    geo_data=districts_with_bars,
    data=districts_with_bars,
    columns=("id", "count_per_district"),
    key_on="feature.id",
    legend_name="Bars per district, Berlin",
)

# add input layer to base layer
districts_with_bars_layer.add_to(interactive_map)


# add tooltipp to display values 
def style_function(feature):
    return {
        "color": "transparent",
        "fillColor": "transparent"
    }

# More complex tooltips can be created using the
# `folium.features.GeoJsonTooltip` class. Below, we use
# its most basic features: `fields` specifies which columns
# should be displayed, `aliases` how they should be labelled.
tooltip = folium.features.GeoJsonTooltip(
    fields=(["BEZIRK_NAM", "count_per_district"]),
    aliases=(["District:", "Bars per district:"])
)

# create tooltipp layer
tooltip_layer = folium.features.GeoJson(
    districts_with_bars,
    style_function=style_function,
    tooltip=tooltip
)

# add tooltipp layer to base layer
tooltip_layer.add_to(interactive_map)

# display map
interactive_map